In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme()

In [2]:
fips_codes = pd.read_csv("Data/FIPS_County_Key.csv", encoding="ISO-8859-1")[['fips','county_name']]
fips_codes.head()

,fips,county_name
0,1001,Autauga County
1,1003,Baldwin County
2,1005,Barbour County
3,1007,Bibb County
4,1009,Blount County


In [3]:
mask_use = pd.read_csv("Data/Mask_Use.csv")
mask_use['mask_use'] = mask_use['FREQUENTLY'] + mask_use['ALWAYS']
mask_use = mask_use[['COUNTYFP','mask_use']]
mask_use.head()

,COUNTYFP,mask_use
0,1001,0.739
1,1003,0.759
2,1005,0.692
3,1007,0.850
4,1009,0.653


In [4]:
fips_mask = pd.merge(fips_codes,mask_use,left_on='fips', right_on='COUNTYFP').drop('COUNTYFP', axis=1)

In [5]:
demographics = pd.read_csv("Data/Demographics.csv").drop(['COUNTY'],axis=1)
demographics['state'] = demographics['STATE'].str.capitalize()
demographics['population'] = demographics['E_TOTPOP']
demographics['poverty'] = demographics['E_POV'] / demographics['E_TOTPOP']
demographics['unemployment'] = demographics['E_UNEMP'] / demographics['E_TOTPOP']
demographics['income'] = demographics['E_PCI']
demographics['elderly'] = demographics['E_AGE65'] / demographics['E_TOTPOP']
demographics['minority'] = demographics['E_MINRTY'] / demographics['E_TOTPOP']
demographics['close_living'] = demographics['E_CROWD'] / demographics['E_TOTPOP']
demographics['uninsured'] = demographics['E_UNINSUR'] / demographics['E_TOTPOP']
demographics = demographics.drop(['E_TOTPOP','E_POV','E_UNEMP','E_PCI','E_AGE65','E_MINRTY','E_CROWD','E_UNINSUR'], axis=1)
demographics.head()

,STATE,FIPS,state,population,poverty,unemployment,income,elderly,minority,close_living,uninsured
0,ALABAMA,1001,Alabama,55200,0.152572,0.019293,29372,0.145833,0.249783,0.005417,0.070199
1,ALABAMA,1009,Alabama,57645,0.142597,0.015769,22656,0.177518,0.128597,0.005881,0.109342
2,ALABAMA,1013,Alabama,20025,0.231710,0.028315,20430,0.190062,0.481448,0.005943,0.100125
3,ALABAMA,1015,Alabama,115098,0.180881,0.040209,24706,0.168430,0.275200,0.006707,0.092843
4,ALABAMA,1017,Alabama,33826,0.163513,0.022852,22827,0.189470,0.442086,0.011943,0.106811


In [6]:
demographics_fips = pd.merge(fips_mask,demographics,left_on='fips', right_on='FIPS').drop(['FIPS','STATE'],axis=1)
demographics_fips.head()

,fips,county_name,mask_use,state,population,poverty,unemployment,income,elderly,minority,close_living,uninsured
0,1001,Autauga County,0.739,Alabama,55200,0.152572,0.019293,29372,0.145833,0.249783,0.005417,0.070199
1,1003,Baldwin County,0.759,Alabama,208107,0.104047,0.020869,31203,0.195404,0.169812,0.004762,0.100256
2,1005,Barbour County,0.692,Alabama,25782,0.255876,0.035606,18461,0.179738,0.538515,0.012140,0.099217
3,1007,Bibb County,0.850,Alabama,22527,0.127092,0.029209,20199,0.162516,0.254184,0.002308,0.071869
4,1009,Blount County,0.653,Alabama,57645,0.142597,0.015769,22656,0.177518,0.128597,0.005881,0.109342


In [7]:
deaths = pd.read_csv("Data/us-counties-deaths.csv")
deaths = deaths[['fips','deaths']]
deaths.head()

,fips,deaths
0,1001.0,102.0
1,1003.0,301.0
2,1005.0,55.0
3,1007.0,58.0
4,1009.0,132.0


In [8]:
deaths_fips = pd.merge(demographics_fips,deaths,on='fips')
deaths_fips.head()

,fips,county_name,mask_use,state,population,poverty,unemployment,income,elderly,minority,close_living,uninsured,deaths
0,1001,Autauga County,0.739,Alabama,55200,0.152572,0.019293,29372,0.145833,0.249783,0.005417,0.070199,102.0
1,1003,Baldwin County,0.759,Alabama,208107,0.104047,0.020869,31203,0.195404,0.169812,0.004762,0.100256,301.0
2,1005,Barbour County,0.692,Alabama,25782,0.255876,0.035606,18461,0.179738,0.538515,0.012140,0.099217,55.0
3,1007,Bibb County,0.850,Alabama,22527,0.127092,0.029209,20199,0.162516,0.254184,0.002308,0.071869,58.0
4,1009,Blount County,0.653,Alabama,57645,0.142597,0.015769,22656,0.177518,0.128597,0.005881,0.109342,132.0


In [9]:
vaccinations = pd.read_csv("Data/Vaccinations_State.csv").groupby("location").max()['people_vaccinated_per_hundred']
vaccinations = vaccinations.drop(['Bureau of Prisons','American Samoa','Dept of Defense','Indian Health Svc','Federated States of Micronesia','Virgin Islands','Long Term Care','Marshall Islands','Northern Mariana Islands','Republic of Palau','Guam','Veterans Health','District of Columbia'])
vaccinations.rename(index={'New York State':'New York'},inplace=True)
vaccinations.head()

location
Alabama       26.34
Alaska        35.63
Arizona       32.68
Arkansas      29.20
California    35.13
Name: people_vaccinated_per_hundred, dtype: float64

In [10]:
all_data = pd.merge(deaths_fips,vaccinations,left_on='state',right_on='location')
all_data

,fips,county_name,mask_use,state,population,poverty,unemployment,income,elderly,minority,close_living,uninsured,deaths,people_vaccinated_per_hundred
0,1001,Autauga County,0.739,Alabama,55200,0.152572,0.019293,29372,0.145833,0.249783,0.005417,0.070199,102.0,26.34
1,1003,Baldwin County,0.759,Alabama,208107,0.104047,0.020869,31203,0.195404,0.169812,0.004762,0.100256,301.0,26.34
2,1005,Barbour County,0.692,Alabama,25782,0.255876,0.035606,18461,0.179738,0.538515,0.012140,0.099217,55.0,26.34
3,1007,Bibb County,0.850,Alabama,22527,0.127092,0.029209,20199,0.162516,0.254184,0.002308,0.071869,58.0,26.34
4,1009,Blount County,0.653,Alabama,57645,0.142597,0.015769,22656,0.177518,0.128597,0.005881,0.109342,132.0,26.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,56037,Sweetwater County,0.414,Wyoming,44117,0.118707,0.027495,32624,0.107011,0.204230,0.006981,0.118775,37.0,29.20
2682,56039,Teton County,0.587,Wyoming,23059,0.070211,0.009107,53703,0.135956,0.184136,0.028535,0.099961,9.0,29.20
2683,56041,Uinta County,0.471,Wyoming,20609,0.123829,0.029793,27009,0.121209,0.124557,0.010626,0.121258,12.0,29.20
2684,56043,Washakie County,0.572,Wyoming,8129,0.121048,0.031123,27556,0.207406,0.179973,0.007381,0.150449,26.0,29.20


In [11]:
mobility21 = pd.read_csv("Data/Mobility2021.csv")
mobility20 = pd.read_csv("Data/Mobility2020.csv")
mobility20 = mobility20.dropna(subset=['census_fips_code'],axis=0)[['census_fips_code','workplaces_percent_change_from_baseline']]
mobility21 = mobility21.dropna(subset=['census_fips_code'],axis=0)[['census_fips_code','workplaces_percent_change_from_baseline']]
mobility = pd.concat([mobility21,mobility20]).groupby('census_fips_code').mean()

In [12]:
all_data = pd.merge(all_data,mobility,left_on='fips',right_on='census_fips_code').dropna()
all_data

,fips,county_name,mask_use,state,population,poverty,unemployment,income,elderly,minority,close_living,uninsured,deaths,people_vaccinated_per_hundred,workplaces_percent_change_from_baseline
0,1001,Autauga County,0.739,Alabama,55200,0.152572,0.019293,29372,0.145833,0.249783,0.005417,0.070199,102.0,26.34,-22.312039
1,1003,Baldwin County,0.759,Alabama,208107,0.104047,0.020869,31203,0.195404,0.169812,0.004762,0.100256,301.0,26.34,-21.561743
2,1005,Barbour County,0.692,Alabama,25782,0.255876,0.035606,18461,0.179738,0.538515,0.012140,0.099217,55.0,26.34,-18.791237
3,1007,Bibb County,0.850,Alabama,22527,0.127092,0.029209,20199,0.162516,0.254184,0.002308,0.071869,58.0,26.34,-19.404639
4,1009,Blount County,0.653,Alabama,57645,0.142597,0.015769,22656,0.177518,0.128597,0.005881,0.109342,132.0,26.34,-23.194103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,56037,Sweetwater County,0.414,Wyoming,44117,0.118707,0.027495,32624,0.107011,0.204230,0.006981,0.118775,37.0,29.20,-18.709184
2453,56039,Teton County,0.587,Wyoming,23059,0.070211,0.009107,53703,0.135956,0.184136,0.028535,0.099961,9.0,29.20,-31.118557
2454,56041,Uinta County,0.471,Wyoming,20609,0.123829,0.029793,27009,0.121209,0.124557,0.010626,0.121258,12.0,29.20,-15.414948
2455,56043,Washakie County,0.572,Wyoming,8129,0.121048,0.031123,27556,0.207406,0.179973,0.007381,0.150449,26.0,29.20,-19.514493
